In [2]:
#importing google drive to mount the required dataset file

from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#importing required libraries

import pandas as pd

import numpy as np

import re

from bs4 import BeautifulSoup

from nltk.stem import PorterStemmer

In [4]:
#Reading dataset(csv file) using pandas:

df = pd.read_csv('/content/drive/My Drive/Reviews.csv')

df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [5]:
#shape of the dataframe:

print('No of rows and columns availabe in dataframe:',df.shape)

No of rows and columns availabe in dataframe: (568454, 10)


In [0]:
#we dont want neutral score (3) datapoints to be present in the dataframe. So we are just taking out those rows from datafraame
filtered_data = df[df['Score']!= 3]

In [7]:
print("No of rows and columns in filtered_data:",filtered_data.shape)

No of rows and columns in filtered_data: (525814, 10)


In [8]:
#We need to get polarity of the reviews from score column :

def partition(x):

  if x<3:

    return 'negative'

  return 'positive'


actual_score = filtered_data['Score']

positivenegative = actual_score.map(partition)

filtered_data['Score'] = positivenegative




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [9]:
#Now we can see the polarity of each review in Score column

filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,positive,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,negative,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,positive,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,positive,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [0]:
#sort dataframe in acending order by using productid column

filtered_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False)

In [11]:
#Deduplication of entries

filtered_data=filtered_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
filtered_data.shape

(364173, 10)

In [12]:
#Condition for  helpfulness numerator and denominator column 


condition_checking = filtered_data[filtered_data['HelpfulnessNumerator'] > filtered_data['HelpfulnessDenominator']]

condition_checking

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,positive,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,positive,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [0]:
#From above we can know there are two rows which has helpfulnesss numertor value high than helfulness denominator value

#remove two rows from dataframe


filtered_data=filtered_data[filtered_data['HelpfulnessNumerator']<=filtered_data['HelpfulnessDenominator']]


In [14]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(filtered_data.shape)

#How many positive and negative reviews are present in our dataset?
filtered_data['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

# **Text Preprocessing**



1.   Remove Punctuations
2.   Remove HTML tags
3.   Remove stopwords
4.   String lowercase 





In [0]:
#Removing html tags from reviews

#Use BeautifulSoup package to remove tags

def remove_htmltags(x):
  example1 = BeautifulSoup(x)
  x = example1.get_text()

  return x

remove_html= filtered_data['Text'].apply(remove_htmltags)

In [0]:
#Adding tgas_removed column to dataframe

pd.options.display.max_colwidth = 1000

filtered_data['tags_removed'] = remove_html


In [0]:
#Removing http links from reviews

def remove_http_links(x):


  output = re.sub(r'https?://\S+', '', x)
  return output

remove_links = filtered_data['tags_removed'].apply(remove_http_links)

In [0]:
#Adding links removed column to dataframe

filtered_data['links_removed'] = remove_links

In [0]:
#Remove numbers(numeric) in reviews:

def remove_numbers(x):
  output= re.sub("\S*\d\S*", "", x).strip()
  return output

remove_num = filtered_data['links_removed'].apply(remove_numbers)



In [0]:
filtered_data['remove_numeric'] = remove_num

In [0]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039

def remove_special(x):

  output = re.sub('[^A-Za-z0-9]+', ' ', x)
  return output

remove_special_characters = filtered_data['remove_numeric'].apply(remove_special)

In [0]:
#Adding final cleaned text data column into dataframe 

filtered_data ['final_text'] = remove_special_characters


In [23]:
#Keeping final_text cleaned data into dataframe and removing rest of the columns added previously


filtered_data.drop(['Text','tags_removed','links_removed','remove_numeric'], axis = 1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,final_text
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at loud i recite it in the car as we re driving along and he always can sing the refrain he s learned about whales India drooping roses i love all the new words this book introduces and the silliness of it all this is a classic book i am willing to bet my son will STILL be able to recite from memory when he is in college
150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version",I grew up reading these Sendak books and watching the Really Rosie movie that incorporates them and love them My son loves them too I do however miss the hard cover version The paperbacks seem kind of flimsy and it takes two hands to keep the pages open
150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their months of the year We will learn all of the poems throughout the school year they like the handmotions which I invent for each poem
150507,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud it has a nice rhythm as well as good repetition that little ones like in the lines about chicken soup with rice The child gets to go through the months of the year and go to wonderful places like Bombay and down the Nile all the while eating well you know what they get to eat Some kids will have Maurice Sendak s version of ice skating or how to treat roses in their heads for a long time and they won t even know where it came from Surprise It came from this little witty book
150508,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of the year It goes through each month and has a cute little poem to go along with it I love this book because it is a really fun way to learn the months and the poems are very creative The author s purpose for writing this book was to give children a fun way to learn the months The children can also learn things about poetry and rhythm through reading this book
...,...,...,...,...,...,...,...,...,...,...
193173,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,positive,1350432000,LOVE!! LOVE!!,LOVE LOVE this sweetener I use it in all my baking my unsweetened flavored DD coffees our unsweetened fruit teas hot cereals anywhere you use regular sugar No calories no carbs no tummy problems no aftertaste no kidding A pork kraut recipe called for apple juice which I didn t have I substituted ZERO drops mixed with apple cider vinegar and no one ever suspected One drop tsp of sugar so it lasts a long time I keep one at home and another in my purse car office
188388,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it It starts out a little sweet with honey then your taste buds become alive with flavors of sweet onion garlic whiskey and a mesquite smoked flavor then just as you are analyzing all the flavors about seconds in you get an awesome habanero kick that comes up in the back side of your mouth that slowly rises in heat about stage hotter than their Sweet Whiskey Habanero sauce but is in no way overwhelming and leaves no bad taste in your mouth Simply perfect I cant wait to try their Smoked Tennessee Maple sauce next
221794,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,positive,1350604800,Awesome Taste,I bought this Hazelnut Paste Nocciola Spread at a local shop in Palm Cost Florida The taste is excellent and healthier than Nutella
5702,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,positive,1351209600,DELICIOUS,Purchased this product at a local sto

# **Featurization : BAG OF WORDS**


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

#Intializing CountVectorizer

count_vect = CountVectorizer()

final_count = count_vect.fit_transform(filtered_data['final_text'].values)

In [25]:
#Shape of the featurized vector


final_count.get_shape()

#we can see totally 118211 features has ben created using for the text column using BAG OF WORDS

(364171, 118211)

In [26]:
#Always BOW matrix will be sparse 

type(final_count)

scipy.sparse.csr.csr_matrix